In [ ]:
import os
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'
print('Device:', DEVICE)

In [ ]:
if os.getcwd() == '/kaggle/working':
    INPUT_DIR = '/kaggle/input/plant-pathology-2020-fgvc7'
else:
    INPUT_DIR = './input'

IMAGE_DIR = os.path.join(INPUT_DIR, 'images')

In [ ]:
train_csv = os.path.join(INPUT_DIR, 'train.csv')
test_cvs = os.path.join(INPUT_DIR, 'test.csv')

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_cvs)

CLASSES = list(train_df.columns[1:])

print('Classes:', CLASSES)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
def f(name, value):
    print("%-20s    %6d" % (name, value))

f('train len:', len(train_df))
f('test len:', len(test_df))
print()
total = 0
for cls in CLASSES:
    s = train_df[cls].sum()
    total += s
    f(cls, s)
f('total sum:', total)

In [ ]:
def read_image_as_numpy(name):
    path = os.path.join(IMAGE_DIR, name+'.jpg')
    tmp = Image.open(path)
    image = np.array(tmp)
    tmp.close()
    return image

In [ ]:
names = list(train_df['image_id']) + list(test_df['image_id'])
shapes = {}
for name in names:
    image = read_image_as_numpy(name)
    n = shapes.get(image.shape, 0)
    shapes[image.shape] = n + 1
print('Shapes:')
for key, value in shapes.items():
    print(key, value)

In [ ]:
nh = 3

def select_by_label(df, name, n):
    series = df[name]
    box = []
    for idx in range(len(df)):
        if series.iloc[idx] == 1:
            box.append(idx)
            if len(box) >= n: break
    return box[:n]


fig, axvh = plt.subplots(len(CLASSES), nh)

fig.set_size_inches(32/2.54, 32/2.54)

for kv, name in enumerate(CLASSES):
    box = select_by_label(train_df, name, nh)
    for kh, idx in enumerate(box):
        image = read_image_as_numpy(train_df.iloc[idx,0])
        title = 'idx=%d  %s' % (idx, name)
        ax = axvh[kv][kh]
        ax.set_title(title)
        ax.axis('off')
        ax.imshow(image)

fig.tight_layout()

plt.show()

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    H = 1365
    W = 2048
    HCROP = 1344
    WCROP = 2016

    def __init__(self, df, size=None, tflag=False):
        super().__init__()
        self.df = df
        self.size = size
        self.tflag = tflag
        
        classes = df.columns[1:]
        if len(classes) == 0:
            self.labels = None
        else:
            v01 = df[classes].values
            self.labels = (v01 * np.array([0,1,2,3])).sum(axis=1)

        box = []
        if size is not None:
            h, w = size
            assert h <= self.HCROP and w <= self.WCROP
            if tflag:
                box.append(A.RandomCrop(self.HCROP, self.WCROP))
            else:
                box.append(A.CenterCrop(self.HCROP, self.WCROP))
            if h != self.H or w != self.W:
                box.append(A.Resize(h, w))       
        if tflag:
            add = [
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.RandomBrightness(p=0.5),
                A.RandomContrast(p=0.5),
                A.RandomGamma(p=0.5),
            ]
            box.extend(add)
        self.transform_numpy = A.Compose(box)
        box.append(A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)))
        box.append(ToTensorV2())
        self.transform_torch = A.Compose(box)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = self.get_oriented_image(idx)
        image = self.transform_torch(image=image)['image']
        if self.labels is None:
            return image
        else:
            return image, self.labels[idx]

    def get_name(self, idx):
        return self.df.iloc[idx, 0]
    
    def get_oriented_image(self, idx):
        name = self.df.iloc[idx, 0]
        image = read_image_as_numpy(name)
        if image.shape[1] < image.shape[0]:
            image = image.transpose(1, 0, 2)
        return image

    def get(self, idx):
        image = self.get_oriented_image(idx)
        image = self.transform_numpy(image=image)['image']
        return image

In [ ]:
ds = ImageDataset(train_df, (224, 336), True)

_, label = ds[1111]
title = 'Augmentation  idx=1111  %s' % CLASSES[label]

fig, ax = plt.subplots(3, 3)
fig.set_size_inches(32/2.54, 24/2.54)

fig.suptitle(title)
for a in ax.ravel():
    a.axis('off')
    a.imshow(ds.get(1234))
fig.tight_layout()

plt.show()